In [5]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment
load_dotenv()
engine = create_engine(f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}")

# Query all needed columns
query = """
SELECT 
    pitcher,
    stand,
    balls,
    strikes,
    inning,
    outs_when_up,
    on_1b,
    on_2b,
    on_3b,
    pitch_type,
    pitch_number,  -- if available
    home_score,
    away_score,
    inning_topbot
FROM pitch_data
WHERE pitch_type IS NOT NULL
"""
df = pd.read_sql(query, engine)


In [6]:
# 1. Encode runners on base
df['runner_on_1b'] = df['on_1b'].notna().astype(int)
df['runner_on_2b'] = df['on_2b'].notna().astype(int)
df['runner_on_3b'] = df['on_3b'].notna().astype(int)
df['runners_on'] = df[['runner_on_1b', 'runner_on_2b', 'runner_on_3b']].sum(axis=1)
df.drop(['on_1b', 'on_2b', 'on_3b'], axis=1, inplace=True)

# 2. Encode batter stand
from sklearn.preprocessing import LabelEncoder
le_stand = LabelEncoder()
df['stand'] = le_stand.fit_transform(df['stand'])  # R=1, L=0

# 3. Pitch count (use pitch_number if available, else compute)
# pitch_number is per-at-bat; if not available, compute per pitcher
if 'pitch_number' not in df.columns or df['pitch_number'].isna().all():
    df['pitch_count'] = df.groupby('pitcher').cumcount()
else:
    df['pitch_count'] = df['pitch_number']

# 4. Compute score differential (positive = pitcher’s team is winning)
df['score_diff'] = df.apply(lambda row: (row['home_score'] - row['away_score']) if row['inning_topbot'] == 'Top' else (row['away_score'] - row['home_score']), axis=1)
df.drop(['home_score', 'away_score', 'inning_topbot'], axis=1, inplace=True)


In [8]:
# Drop rows with NaN and keep only features needed
df.dropna(inplace=True)
df_final = df[['pitcher', 'stand', 'balls', 'strikes', 'inning', 'outs_when_up', 'runners_on', 'pitch_count', 'score_diff', 'pitch_type']]
df_final.to_csv("../data/processed_features.csv", index=False)


In [13]:
# grouping plan

pitch_type_groups = {
    'FF': 'Fastball',
    'FT': 'Fastball',
    'SI': 'Fastball',
    'FC': 'Fastball',
    'FS': 'Fastball',      # Split-finger fastball
    'SL': 'Breaking',
    'ST': 'Breaking',      # Sweeper
    'CU': 'Breaking',
    'KC': 'Breaking',      # Knuckle Curve
    'CH': 'Offspeed',
    'SC': 'Offspeed',      # Screwball
    'KN': 'Other',
    'EP': 'Other',
    'FO': 'Other'
}


In [9]:
# === Group pitch types into broader categories ===

pitch_type_map = {
    'FF': 'Fastball', 'FT': 'Fastball', 'SI': 'Fastball', 'FC': 'Fastball', 'FS': 'Fastball',
    'SL': 'Breaking', 'ST': 'Breaking', 'CU': 'Breaking', 'KC': 'Breaking',
    'CH': 'Offspeed', 'SC': 'Offspeed',
    'KN': 'Other', 'EP': 'Other', 'FO': 'Other'
}

# Map to new column
df['pitch_type_group'] = df['pitch_type'].map(pitch_type_map)

# Drop rows with missing or unmapped pitch types
df = df.dropna(subset=['pitch_type_group'])

# Reorder columns (optional)
cols = [c for c in df.columns if c != 'pitch_type_group'] + ['pitch_type_group']
df = df[cols]

# Save the updated dataframe
df.to_csv("../data/processed_features.csv", index=False)
print("✅ Saved processed_features.csv with pitch_type_group column")


✅ Saved processed_features.csv with pitch_type_group column


In [12]:
df.head(50)

,pitcher,stand,balls,strikes,inning,outs_when_up,pitch_type,pitch_number,runner_on_1b,runner_on_2b,runner_on_3b,runners_on,pitch_count,score_diff,pitch_type_group
0,687863,1,1,1,9,1,ST,3,0,0,0,0,3,3,Breaking
1,687863,1,0,1,9,1,ST,2,0,0,0,0,2,3,Breaking
2,687863,1,0,0,9,1,ST,1,0,0,0,0,1,3,Breaking
3,687863,0,0,1,9,0,FF,2,0,0,0,0,2,3,Fastball
4,687863,0,0,0,9,0,ST,1,0,0,0,0,1,3,Breaking
5,660813,1,0,2,9,2,FF,3,1,1,0,2,3,-3,Fastball
6,660813,1,0,1,9,2,SI,2,1,1,0,2,2,-3,Fastball
7,660813,1,0,0,9,2,SI,1,1,1,0,2,1,-3,Fastball
8,660813,0,2,1,9,2,SI,4,1,0,0,1,4,-3,Fastball
9,660813,0,1,1,9,2,FC,3,1,0,0,1,3,-3,Fastball
